<a href="https://colab.research.google.com/github/Hemanthnaidu3737/AI-Document-Analysis/blob/main/BERT_SENTIMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio transformers torch


In [ ]:
import gradio as gr
from transformers import pipeline
import torch
import time
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# Enable GPU if available
device = 0 if torch.cuda.is_available() else -1

In [ ]:
# Keyword lists
positive_keywords = [
    "happy", "joy", "love", "awesome", "great", "fantastic", "thank you",
    "excited", "peace", "satisfied", "grateful", "amazing", "cool", "positive",
    "brilliant", "nice", "sweet", "adorable", "hopeful", "cheerful", "blessed",
    "like", "hope", "smile", "wonderful", "excellent", "lovely", "beautiful",
    "friendly", "enjoy", "delightful", "fabulous", "nice work", "well done",
    "encouraging", "proud", "motivated", "inspired", "optimistic", "uplifting",
    "joyful", "awesome job", "best", "glad", "thankful", "victory", "success",
    "charming", "supportive", "respectful", "calm", "golden", "celebrate", "champion",
    "content", "peaceful", "harmony", "radiant", "energetic", "generous", "lucky",
    "win", "progress", "growth", "fun", "genuine", "warm", "affection", "devoted",
    "smiling", "kind", "loving", "appreciated", "healing", "strong", "determined",
    "confident", "playful", "admirable", "loyal", "mindful", "caring", "heartwarming",
    "positive vibes", "good job", "you're amazing", "way to go", "phenomenal",
    "outstanding", "you rock", "sweetheart", "heartfelt", "pleased", "bright", "upbeat",
    "graceful", "dream big", "be proud", "joy inside"
]



negative_keywords = [
    "sad", "lonely", "hopeless", "depressed", "anxious", "afraid", "angry", "hate",
    "furious", "suicidal", "kill myself", "die", "worthless", "failure", "panic",
    "murder", "crime", "violence", "hurt", "awful", "terrible", "pain", "suffering",
    "burnout", "exhausted", "guilty", "ashamed", "cut", "self-harm", "frustrated",
    "suffer", "f*ck", "sh*t", "broken", "crying", "giving up", "hate myself", "numb",
    "despair", "stressed", "rejected", "abandoned", "unloved", "useless", "can’t take it",
    "end it all", "overwhelmed", "mental breakdown", "trapped", "down", "devastated", "rude",
    "hopelessness", "darkness", "rage", "fear", "tears", "dying", "lost", "empty",
    "helpless", "no one cares", "screaming", "unworthy", "heartbroken", "breakdown",
    "isolation", "destructive", "lifeless", "worthless piece", "hate everyone",
    "no reason to live", "what's the point", "not enough", "nobody", "alone", "abuse",
    "neglected", "trauma", "shattered", "toxic", "downfall", "worse", "defeated",
    "burden", "sorrow", "cry", "low", "messed up", "can't breathe", "hurting",
    "emotional pain", "beyond help", "dark thoughts", "unfixable", "hell", "ugly",
    "insecure", "shame", "regret", "can't cope", "falling apart", "unseen", "ignored",
    "blame", "failure again", "rejected again", "numbness", "nightmare"
]



neutral_keywords = [
    "okay", "alright", "fine", "neutral", "average", "normal", "typical", "standard",
    "maybe", "not sure", "possibly", "perhaps", "hmm", "k", "noted", "whatever",
    "doesn’t matter", "no opinion", "sometimes", "depends", "neither", "in between",
    "it is what it is", "okayish", "meh", "so-so", "I guess", "more or less", "generally",
    "can you", "how to", "what if", "what about", "not bad", "nothing special", "nothing much",
    "informative", "understood", "not important", "it's fine", "acceptable", "moderate",
    "decent", "typical day", "regular", "same as usual", "just wondering", "as expected",
    "let's see", "not a big deal", "satisfactory", "unclear", "ambiguous", "somehow",
    "uncertain", "maybe later", "eventually", "I don’t know", "could be", "probably",
    "in general", "for now", "let me think", "guess so", "alright then", "heard that",
    "reasonable", "I suppose", "tolerable", "low-key", "casual", "no change", "unchanged",
    "not really", "little change", "non-emotional", "regular update", "status quo",
    "can’t say", "more or less fine", "typical response", "just saying", "on hold",
    "need details", "just checking", "waiting", "need more info", "technical", "background info",
    "not now", "perhaps later", "no rush", "sometime", "pause", "in progress", "partial",
    "random", "neutral tone", "some updates", "non-specific", "okay response", "so be it"
]

suicidal_keywords = [
    "suicidal", "i'm suicidal", "i am suicidal", "want to die", "i want to die", "want to end it", "i want to end it",
    "end my life", "i want to end my life", "i will kill myself", "i'm going to kill myself", "going to kill myself",
    "i will end it", "i'm done", "i'm finished", "i can't go on", "can't go on", "can't take it anymore", "can't take it",
    "i want to die alone", "i want to disappear", "i wish i were dead", "wish i were dead", "wish i'd never been born",
    "i wish i'd never been born", "i don't want to live", "don't want to live", "there's no point", "what's the point",
    "no reason to live", "no reason to continue", "end it all", "end it now", "time to end it", "i'm going to end it",
    "i'm going to die", "i will die", "please just let me die", "i'm done with life", "done with life", "i can't do this anymore",
    "i want out", "i want out of this life", "i want to leave this world", "leave this world", "take my life",
    "take my own life", "i'm going to hurt myself", "i might hurt myself", "i'm going to give up", "i give up on life",
    "i'm giving up", "i want to disappear forever", "don't want to be here", "can't stay alive", "going to end my life",
    "i don't care if i die", "i hope i die", "wish i would die", "hope i die", "i deserve to die", "should just die",
    "shouldn't be alive", "not worth living", "worthless", "worthless life", "i am worthless", "life is pointless",
    "life is meaningless", "no one cares if i die", "no one would miss me", "no one would care", "nobody cares",
    "i am better off dead", "better off dead", "maybe it's better if i die", "i can't handle this", "i can't cope",
    "can't cope anymore", "i'm falling apart", "falling apart", "broken beyond repair", "beyond help", "beyond saving",
    "there's no help", "i will never get better", "don't want to wake up", "don't want to wake up tomorrow",
    "wish i wouldn't wake up", "sleep forever", "permanent sleep", "i'm finished with everything", "i can't bear this pain",
    "can't bear the pain", "sick of living", "tired of living", "tired of life", "tired of everything", "exhausted of living",
    "just want to end it", "just want it to be over", "let it all end", "this is unbearable", "i can't do this pain",
    "i'm beyond saving", "there's no way out", "no escape from pain", "alone and hopeless", "completely hopeless", "hopelessness",
    "seeking an exit", "need an exit", "done with myself", "ready to die", "prepared to die", "ready to end it", "lost all hope",
    "completely lost", "despairing", "in despair", "desperate to die", "thinking about suicide", "thoughts of suicide",
    "i have suicidal thoughts", "planning to kill myself", "i'm planning to kill myself", "i'm making a plan to die",
    "i want to end my suffering", "end my suffering", "life is a burden", "burden to others", "i am a burden",
    "can't face tomorrow", "don't want tomorrow", "don't want to be here tomorrow", "please let me die",
    "i'm hopelessly depressed", "deeply depressed", "so depressed i want to die", "want to leave this world",
    "no future for me", "no future", "ready to give up", "ready to quit life", "give up on life", "ready to walk away",
    "i need to disappear", "fade away", "wish i could vanish", "vanish forever", "losing the will to live",
    "lost the will to live", "lost will to live", "want to stop existing"
]

import random

intent_responses = {
    "positive": [
        "That's great to hear! 😊",
        "Awesome! Keep spreading positivity! 🎉"
    ],
    "neutral": [
        "Okay, got it! 😐",
        "Thanks for sharing!"
    ],
    "negative": [
        "I'm sorry to hear that. 😔",
        "Let's try to make things better! 🌟"
    ],
    "suicidal": [
        "I’m really concerned about your safety. You’re not alone. Please reach out to someone you trust or a trained professional. 💛",
        "It sounds like you’re going through a very tough time. If you feel unsafe, please contact a helpline immediately: KIRAN (India): 1800-599-0019, 988 (USA), Samaritans (UK): 116 123.",
        "Your feelings matter. Please take a moment to talk to a trained counselor or call an emergency helpline. You can stay safe. 💛"
    ]
}


def get_intent_response(user_text):
    user_text_lower = user_text.lower()

    # Check suicidal keywords first
    if any(word in user_text_lower for word in suicidal_keywords):
        return random.choice(intent_responses["suicidal"])

    # Check negative keywords
    elif any(word in user_text_lower for word in negative_keywords):
        return random.choice(intent_responses["negative"])

    # Check positive keywords
    elif any(word in user_text_lower for word in positive_keywords):
        return random.choice(intent_responses["positive"])

    # Default neutral
    else:
        return random.choice(intent_responses["neutral"])

In [ ]:
# Load pre-trained sentiment analysis model
sentiment_pipeline = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
intent_responses = {
    "positive": [
        "That's great to hear! 😊",
        "Awesome! Keep spreading positivity! 🎉"
    ],
    "neutral": [
        "Okay, got it! 😐",
        "Thanks for sharing!"
    ],
    "negative": [
        "I'm sorry to hear that. 😔",
        "Let's try to make things better! 🌟"
    ],
    "suicidal": [
        "I’m really concerned about your safety. You’re not alone. Please reach out to someone you trust or a trained professional. 💛",
        "It sounds like you’re going through a very tough time. If you feel unsafe, please contact a helpline immediately: KIRAN (India): 1800-599-0019, 988 (USA), Samaritans (UK): 116 123.",
        "Your feelings matter. Please take a moment to talk to a trained counselor or call an emergency helpline. You can stay safe. 💛"
    ]
}


In [ ]:
# Function to predict sentiment and respond
def predict_sentiment(user_input):
    result = sentiment_pipeline(user_input)
    label = result[0]["label"]

    # Map model labels to positive, neutral, negative
    if "5 stars" in label or "4 stars" in label:
        sentiment = "positive"
    elif "3 stars" in label:
        sentiment = "neutral"
    else:
        sentiment = "negative"

    # Pick a response from the predefined list
    response = intent_responses[sentiment][0]

    return f"Sentiment: {sentiment.capitalize()}\nResponse: {response}"

In [ ]:
interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=3, placeholder="How are you feeling today?"),
    outputs="text",  # <- Fix here
    title="💬 Sentiment Analysis using BERT and Transformers",
    description="An AI-powered sentiment analysis tool using BERT and Transformers to detect emotional states and provide supportive responses. ❤️"
)


In [ ]:
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5f9cd4c8d0822d8285.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
